In [19]:
import torch

from transformers import BertTokenizer
from PIL import Image
import argparse

from catr.models import caption
from catr.datasets import coco, utils
from catr.configuration import Config
import os

image_path = 'test/0.jpg'
version = 'v3'

config = Config()

if version == 'v1':
    model = torch.hub.load('saahiluppal/catr', 'v1', pretrained=True)
elif version == 'v2':
    model = torch.hub.load('saahiluppal/catr', 'v2', pretrained=True)
elif version == 'v3':
    model = torch.hub.load('saahiluppal/catr', 'v3', pretrained=True)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

image = Image.open(image_path)
image = coco.val_transform(image)
image = image.unsqueeze(0)


def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template


caption, cap_mask = create_caption_and_mask(
    start_token, config.max_position_embeddings)


@torch.no_grad()
def evaluate(image):
    print('wait')
    model.eval()
    for i in range(config.max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption


#output = evaluate(image)
#result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
#print(result.capitalize())

Using cache found in /root/.cache/torch/hub/saahiluppal_catr_master


In [42]:
def predict(image_path):
    image = Image.open(image_path)
    image = coco.val_transform(image)
    image = image.unsqueeze(0)
    for i in range(config.max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return tokenizer.decode(caption[0].tolist(), skip_special_tokens=True)

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False
    return tokenizer.decode(caption[0].tolist(), skip_special_tokens=True)

In [45]:
predict('test/18.jpg')

'a man is jumping over a wave in a body of water..'